>Ouvrir le notebook dans Colab en modifiant le début de son adresse dans le navigateur :<br>
il faut remplacer **github.com** par **githubtocolab.com**.<br>
Une fois vos réponses apportées, le notebook devra être sauvegardé dans GitHub, dans le repository du TP :<br>
*Fichier > Sauvegarder une copie dans Github*<br>

---

# Un chouïa d'apprentissage automatique (machine learning)

On récupère les données telles qu'on les a laissées à la fin du notebook `TP3a`.

In [ ]:
# imports
import pandas as pd # bibliothèques dédiée au traitement de jeux de données
import matplotlib.pyplot as plt # bibliothèque graphique
import seaborn as sns # bibliothèque graphique reposant sur matplotlib et dédiée plus particulièrement à la représentation de jeux de données
import numpy as np # bibliothèque puissante permettant de gérer des tableaux multidimensionnels
import plotly.express as px # libraire permettant des graphes interactifs
import plotly.graph_objects as go # complémentaire à la première

# paramètres par défaut pour le graphes
plt.rcParams['figure.figsize'] = (15, 6)
plt.rcParams['font.family'] = "serif"
plt.rcParams['font.size'] = 13
sns.set_style("white")

# données
url = "https://raw.githubusercontent.com/Info-TSI-Vieljeux/s1-tp3/main/data_monde_groupe.csv"
data_monde = pd.read_csv(url,index_col=0)
data_monde.head(3)

On a vu qu'un regroupement des données en 3 grands groupes "Nord", "Sud" et "Intermédiaire" semble plutôt cohérent.<br>
Mais pourquoi pas laisser un algorithme décider lui-même de qui va le mieux ensemble ? Ensuite nous pourrons vérifier si cela recoupe notre découpage fait à la main.<br>
On appelle cela un **apprentissage non supervisé**.

Nous allons utiliser l'algorithme des **k-moyennes** pour partitionner automatiquement nos données.<br>
Il consiste à placer chaque point de données dans un espace à $n$ dimensions où $n$ est le nombre de variables (les descripteurs) et chercher à les regrouper en **clusters** en fonction de leurs distances.<br>
Chaque variable correspondant à un axe du repère. 

Pour aider l'algorithme, on peut tenter de réduire la dimension de l'espace dans lequel chaque point de données est plongé en utilisant une **analyse en composantes principales**.<br>
L'idée est de déterminer les combinaisons des différentes variables expliquant le mieux la variance des données. Chaque nouvel axe ainsi formé (les composantes principales) explique une part décroissante mais complémentaire de la variance (sur la deuxième composante, les données sont moins étalées que sur la première, mais elles s'étalent dans une direction orthogonale, et ainsi de suite).<br>
Projeter les données sur les premières composantes permet d'éclater le plus possible les données. On peut ainsi réduire l'espace à *n* dimensions du départ à un espace de seulement 2 ou 3 dimensions expliquant la majorité de la variance des données.

La bibliothèque `Scikit-learn`, destinée à l'apprentissage automatique, contient tout ce qu'il nous faut :

In [ ]:
from sklearn.decomposition import PCA # l'algorithme d'analyse en composantes principales (PCA en anglais)
from sklearn.preprocessing import StandardScaler # pour centrer-réduire les données
from sklearn.cluster import KMeans # l'algorithme des k-moyennes

In [ ]:
variables = data_monde.columns.values[1:-1]
scaler = StandardScaler()
X = scaler.fit_transform(data_monde[variables]) 
# chaque vecteur correspondant à chacune des variables est maintenant centré-réduit
pca = PCA()
components = pca.fit_transform(X)

Quelle combinaison des variables de départ utilise la première composante ?

In [ ]:
data = data_monde.copy() # pour pouvoir revenir sur le graphe suivant même après ajout de colonnes à data_monde

In [ ]:
n_c = 1 # numéro de la composante principale à décrire
px.bar(components.T, x=data.columns.values[1:-1], y=n_c-1, labels={f"{n_c-1}": f"Composante Principale (CP) {n_c}"})

Quelle est le nom de la variable participant le plus à la composante principale n°34 ?

In [ ]:
# noter son nom tel qu'il apparaît dans les données
variable = "nom"

In [ ]:
# Cellule de vérification (ne pas modifier)

Représentons le pourcentage de variance expliqué par chacune des composantes :

In [ ]:
exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)
fig = px.bar(x=range(1, exp_var_cumul.shape[0] + 1),y=pca.explained_variance_ratio_,labels={"x": "composante", "y": "% variance expliquée"})
fig.add_scatter(x=list(range(1, exp_var_cumul.shape[0] + 1)), y=exp_var_cumul, name="", showlegend=False)

Les trois premières composantes expliquent presque 90% de la variance !<br>
Plaçons les données dans un espace réduit à ces 3 dimensions :

In [ ]:
px.scatter_3d(components, x=0, y=1, z=2, 
              color=data_monde['Groupe'],
              labels={'0': 'CP 1', '1': 'CP 2', '2': 'CP 3'},
              hover_name=data_monde.index)

On constate à nouveau que nos 3 groupes discriminent plutôt très bien nos données même si quelques chevauchements existent.<br>
C'est le moment d'utiliser l'algorithme des k-moyennes pour essayer de former 3 groupes homogènes :

In [ ]:
# on ne garde qe les 3 premières composantes principales
pca = PCA(n_components = 3)
pca.fit(X)
score_pca = pca.transform(X)
kmeans_pca = KMeans(n_clusters=3,init='k-means++',random_state=42)
kmeans_pca.fit(score_pca)
data_monde["Cluster"]=kmeans_pca.labels_.astype(str)
data_monde.head(3)

In [ ]:
fig = px.scatter_3d(components, x=0, y=1, z=2, 
              color=data_monde['Cluster'],
              labels={'0': 'CP 1', '1': 'CP 2', '2': 'CP 3'},
              color_discrete_sequence=px.colors.qualitative.Bold,
              hover_name=data_monde.index)
fig.update_layout(legend_title = "Cluster")

Les 3 clusters créés reproduisent à peu de chose près les 3 groupes "Nord", "Sud", "Intermédiaire" construits à la main.<br>
À quel cluster correspondent approximativement les pays du groupe "Nord" ?

In [ ]:
# noter le numéro entre les guillements
cluster = "..."

In [ ]:
# Cellule de vérification (ne pas modifier)

Mais l'accord n'est pas parfait !<br> 
Citer un pays qui appartient au groupe "Nord" mais qui n'appartient pas au cluster lui correspondant.

In [ ]:
# noter le nom tel qu'il est écrit dans les données
nom_du_pays = "Pays"

In [ ]:
# Cellule de vérification (ne pas modifier)

Nous allons voir dans la dernière partie du TP comment représenter ces données sur une carte pour y voir plus clair.

In [ ]:
data_monde.to_csv('data_monde_classe.csv')